# Введение в MapReduce модель на Python


In [48]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [49]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [50]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [51]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [52]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [53]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [54]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [55]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [56]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [57]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [58]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [59]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [60]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [4]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

NameError: name 'MapReduce' is not defined

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [61]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [62]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18), ('it', 18)]),
 (1, [('a', 2), ('what', 10)])]

## TeraSort

In [63]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.028133559580729384),
   (None, 0.09025289233963807),
   (None, 0.12438930448150842),
   (None, 0.1617237910066578),
   (None, 0.21446275627401656),
   (None, 0.2553431575210646),
   (None, 0.25644520433055773),
   (None, 0.2659223878267921),
   (None, 0.280137399137223),
   (None, 0.30153159801758944),
   (None, 0.3816045814017959),
   (None, 0.4141810556445422),
   (None, 0.42429844878478584),
   (None, 0.4933592975783354)]),
 (1,
  [(None, 0.5293027082813384),
   (None, 0.5920048802709277),
   (None, 0.5989743538130635),
   (None, 0.6127018384718558),
   (None, 0.623020203116356),
   (None, 0.6376999274910841),
   (None, 0.6678618208980652),
   (None, 0.7262232089021787),
   (None, 0.7378522048892291),
   (None, 0.7610477670975955),
   (None, 0.7633752506177925),
   (None, 0.7649883686324094),
   (None, 0.785784451838158),
   (None, 0.794854450921251),
   (None, 0.8183856159741131),
   (None, 0.9883467798075342)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [65]:
import random
from functools import reduce

def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

def MAP(value):
    yield (value, value)

def REDUCE(max1, max2):
    return max(max1, max2)

record = RECORDREADER(5)
print("Исходные данные:", record)

mapped_values = []
for value in record:
    for k, v in MAP(value):
        mapped_values.append(v)

result = reduce(REDUCE, mapped_values)

print("Максимум:", result)

Исходные данные: [41, 89, 96, 67, 91]
Максимум: 96


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [67]:
import random
from typing import Iterator, Tuple

def MAP(num):
    yield ("all", num)

def REDUCE(key, nums: Iterator[Tuple[str, int]]):
    sum = 0
    count = 0
    for _, n in nums:
        sum += n
        count += 1
    yield (key, sum / count)

def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

record = RECORDREADER(5)

map_output = []
for num in record:
    for k, v in MAP(num):
        map_output.append((k, v))

print("Данные:", map_output)

grouped_data = {}
for k, v in map_output:
    if k not in grouped_data:
        grouped_data[k] = []
    grouped_data[k].append((k, v))

reduce_output = []
for key, values in grouped_data.items():
    for result in REDUCE(key, values):
        reduce_output.append(result)

print("Среднее арифметическое:", reduce_output[0][1])

Данные: [('all', 14), ('all', 67), ('all', 9), ('all', 59), ('all', 40)]
Среднее арифметическое: 37.8


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [80]:
import random
from typing import Iterator, NamedTuple, List, Tuple

def MAP(num):
    return (1, num)

def REDUCE(key, nums: Iterator[int]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count

def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

def group_by_key(iterable: List[Tuple[int, int]]):
    grouped_data = {}
    for key, value in sorted(iterable, key=lambda x: x[0]):
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

map_out = list(map(lambda x: MAP(x), RECORDREADER(10)))
print("MAP:", map_out)

shuffle_out = group_by_key(map_out)
print("Shuffle:", shuffle_out)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_out)))
print("Reduce:", reduce_output)

MAP: [(1, 93), (1, 62), (1, 3), (1, 47), (1, 50), (1, 85), (1, 78), (1, 26), (1, 95), (1, 41)]
Shuffle: [(1, [93, 62, 3, 47, 50, 85, 78, 26, 95, 41])]
Reduce: [58.0]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [100]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    grouped_data = {}
    for key, value in iterable:
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for key, value in map_partition:
            p = partitions[PARTITIONER(key)]
            p.setdefault(key, []).append(value)
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group),
                                                                                    reduce_partition[1]))), reduce_partitions)
    return reduce_outputs
d1 = """
hello it is me
hello it is me
hello it is me"""
d2 = """
hello it is who
hello it is who"""
d3 = """
it is a alexey"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for (docid, document) in enumerate(split):
            for (lineid, line) in enumerate(document.split('\n')):
                if line.strip():
                    yield ("{}:{}".format(docid, lineid), line)

    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i:i + split_size])

def MAP(docId: str, line: str):
    for word in line.split(" "):
        if word.strip():
            yield (word, None)

def REDUCE(key: str, values: Iterator[str]):
    yield key

def COMBINER(key, values):
    yield (key, None)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=COMBINER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

[(0, ['alexey', 'hello', 'is', 'it', 'me', 'who']), (1, ['a'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [105]:
from collections import defaultdict
import random
def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):
            mapped_result[t].append(t)
    return mapped_result.items()
def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)
    for values in mapped_items:
        for value in values:
            reduced_result.append(value)
    return reduced_result
def C(t):
    return t[0] % 2 == 0
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]
record = RECORDREADER(5)
print(record)
part_count = 5
record_partitioned = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitioned)
print(REDUCE(list(map(lambda x: MAP(x), record_partitioned))))

[(0, 24, 2), (78, 45, 71), (10, 16, 33), (77, 13, 62), (62, 1, 59)]
[[(0, 24, 2), (78, 45, 71), (10, 16, 33), (77, 13, 62), (62, 1, 59)]]
[dict_items([((0, 24, 2), [(0, 24, 2)]), ((78, 45, 71), [(78, 45, 71)]), ((10, 16, 33), [(10, 16, 33)]), ((62, 1, 59), [(62, 1, 59)])])]
[((0, 24, 2), [(0, 24, 2)]), ((78, 45, 71), [(78, 45, 71)]), ((10, 16, 33), [(10, 16, 33)]), ((62, 1, 59), [(62, 1, 59)])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [110]:
S = set([148, 66, 63])

def MAP(t):
    res_list = []
    for el in t:
        if el in S:
            res_list.append(el)
    res = tuple(res_list)
    return (res, res)

def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(5)
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print("MAP:", map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print("Shuffle:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print("Reduce:", reduce_output)

[(37, 45, 63), (8, 41, 67), (98, 29, 4), (65, 47, 10), (18, 18, 92)]
MAP: [((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((63,), (63,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((63,), (63,)), ((), ()), ((), ()), ((), ()), ((63,), (63,)), ((), ()), ((), ()), ((), ()), ((63,), (63,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), (

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [114]:
def MAP(t):
    return (t, t)

def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(5)
print(record)

map_out = list(map(lambda x: MAP(x), RECORDREADER(100)))
print("MAP:", map_out)

shuffle_out = group_by_key(map_out)
shuffle_out = list(shuffle_out)
print("Shuffle:", shuffle_out)

reduce_out = list(map(lambda x: REDUCE(*x)[0], shuffle_out))
print("Reduce:", reduce_out)

[(97, 75, 82), (40, 14, 100), (39, 17, 28), (73, 59, 20), (40, 20, 16)]
MAP: [((89, 54, 95), (89, 54, 95)), ((63, 49, 58), (63, 49, 58)), ((30, 82, 11), (30, 82, 11)), ((65, 87, 69), (65, 87, 69)), ((26, 53, 57), (26, 53, 57)), ((57, 35, 58), (57, 35, 58)), ((43, 82, 1), (43, 82, 1)), ((75, 20, 51), (75, 20, 51)), ((77, 60, 20), (77, 60, 20)), ((74, 7, 52), (74, 7, 52)), ((58, 16, 11), (58, 16, 11)), ((41, 32, 90), (41, 32, 90)), ((47, 85, 37), (47, 85, 37)), ((27, 65, 53), (27, 65, 53)), ((22, 10, 59), (22, 10, 59)), ((92, 26, 75), (92, 26, 75)), ((11, 34, 21), (11, 34, 21)), ((49, 93, 11), (49, 93, 11)), ((14, 96, 91), (14, 96, 91)), ((56, 27, 23), (56, 27, 23)), ((52, 29, 51), (52, 29, 51)), ((43, 41, 32), (43, 41, 32)), ((93, 32, 44), (93, 32, 44)), ((62, 95, 97), (62, 95, 97)), ((95, 90, 88), (95, 90, 88)), ((5, 51, 82), (5, 51, 82)), ((45, 42, 46), (45, 42, 46)), ((20, 70, 63), (20, 70, 63)), ((27, 49, 53), (27, 49, 53)), ((0, 64, 93), (0, 64, 93)), ((6, 76, 3), (6, 76, 3)), ((12

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [119]:
def MAP(t):
    return (t, t)

def REDUCE(key, values: Iterator[NamedTuple]):
    if len(values) == 2:
        return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(100)
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print("MAP:", map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
print("Shuffle:", shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print("Reduce:", reduce_output)

[(0, 2), (1, 2), (0, 2), (3, 0), (0, 0), (2, 0), (0, 1), (2, 1), (3, 2), (1, 0), (3, 2), (1, 3), (0, 1), (2, 3), (1, 3), (2, 2), (3, 2), (2, 2), (1, 2), (1, 3), (0, 3), (0, 0), (0, 0), (3, 2), (0, 1), (1, 2), (3, 0), (3, 2), (3, 1), (3, 0), (2, 0), (0, 2), (2, 2), (3, 1), (3, 2), (3, 0), (1, 2), (0, 2), (0, 1), (3, 2), (2, 3), (1, 2), (3, 0), (0, 2), (3, 3), (0, 2), (3, 0), (3, 1), (2, 3), (3, 0), (2, 0), (0, 2), (2, 2), (2, 3), (1, 3), (2, 2), (2, 0), (2, 2), (3, 2), (1, 1), (3, 1), (0, 1), (2, 1), (3, 2), (2, 1), (2, 0), (0, 3), (0, 0), (1, 2), (2, 1), (0, 0), (2, 3), (2, 0), (2, 1), (1, 1), (2, 2), (3, 2), (1, 3), (3, 1), (3, 3), (1, 3), (0, 3), (0, 3), (3, 2), (1, 3), (0, 1), (2, 3), (0, 3), (1, 2), (3, 2), (1, 1), (1, 1), (1, 3), (2, 2), (1, 3), (0, 1), (0, 3), (1, 2), (2, 1), (3, 2)]
MAP: [((2, 3), (2, 3)), ((3, 1), (3, 1)), ((1, 2), (1, 2)), ((1, 0), (1, 0)), ((3, 2), (3, 2)), ((0, 1), (0, 1)), ((2, 1), (2, 1)), ((1, 3), (1, 3)), ((1, 3), (1, 3)), ((0, 3), (0, 3)), ((3, 0), (3, 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [121]:
rels = [1, 2]

class Tuple:
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple(count):
    data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
    rel_id = rels[random.randint(0, len(rels) - 1)]
    return Tuple(data, rel_id)

def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]

def MAP(t: Tuple):
    return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
    if values == [rels[0]]:
        return (key, key)

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(100)

map_out = list(map(lambda x: MAP(x), RECORDREADER(100)))
print("MAP:", map_out)

shuffle_out = group_by_key(map_out)
shuffle_out = list(shuffle_out)
print("Shuffle:", shuffle_out)

reduce_out = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_out)) if el is not None]
print("Reduce:", reduce_out)

MAP: [(((1, 0), (1, 0), (0, 1)), 1), (((2, 1), (3, 1), (2, 2)), 2), (((1, 3), (1, 3), (3, 2)), 1), (((1, 0), (2, 3), (1, 1)), 1), (((3, 1), (2, 3), (1, 2)), 1), (((0, 2), (1, 2), (3, 3)), 2), (((2, 1), (3, 3), (3, 3)), 1), (((3, 0), (3, 2), (2, 0)), 1), (((3, 1), (3, 1), (0, 0)), 1), (((0, 2), (3, 1), (3, 0)), 2), (((2, 1), (0, 1), (1, 0)), 1), (((3, 0), (1, 3), (0, 1)), 2), (((2, 2), (1, 3), (1, 0)), 1), (((0, 1), (2, 0), (2, 1)), 1), (((0, 0), (1, 3), (2, 3)), 1), (((1, 0), (1, 1), (0, 3)), 1), (((0, 1), (3, 2), (0, 0)), 1), (((1, 2), (3, 2), (3, 3)), 2), (((0, 1), (2, 2), (0, 3)), 2), (((1, 1), (2, 3), (2, 1)), 2), (((3, 3), (2, 1), (1, 3)), 1), (((2, 3), (1, 0), (0, 2)), 2), (((3, 2), (2, 3), (3, 3)), 2), (((1, 3), (1, 1), (1, 3)), 2), (((3, 3), (3, 0), (2, 3)), 2), (((1, 0), (1, 1), (0, 3)), 1), (((2, 1), (2, 1), (0, 0)), 2), (((0, 2), (1, 0), (1, 0)), 1), (((3, 2), (3, 0), (1, 3)), 2), (((0, 1), (1, 1), (2, 2)), 1), (((2, 1), (2, 1), (2, 2)), 1), (((1, 2), (3, 3), (3, 2)), 2), ((

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [124]:
rels = [1, 2]

class Tuple:
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)

map_out = list(map(lambda x: MAP(x), RECORDREADER(100)))
print("MAP:", map_out)

shuffle_out = group_by_key(map_out)
shuffle_out = list(shuffle_out)
print("Shuffle:", shuffle_out)

reduce_out = list(map(lambda x: REDUCE(*x), shuffle_out))
print("Reduce:", reduce_out)

MAP: [(0, (2, 2)), (1, (2, 1)), (1, (2, 1)), (3, (1, 2)), (2, (1, 0)), (0, (2, 2)), (0, (2, 3)), (1, (2, 2)), (1, (1, 0)), (3, (1, 3)), (2, (2, 3)), (1, (1, 1)), (1, (1, 1)), (2, (2, 3)), (3, (2, 1)), (0, (1, 3)), (1, (1, 0)), (2, (1, 1)), (1, (1, 0)), (2, (2, 1)), (0, (2, 1)), (2, (2, 2)), (2, (2, 2)), (1, (2, 3)), (3, (1, 3)), (2, (1, 1)), (3, (2, 2)), (3, (2, 2)), (0, (1, 0)), (3, (2, 1)), (2, (2, 3)), (1, (2, 0)), (3, (2, 1)), (1, (2, 3)), (2, (1, 3)), (1, (2, 0)), (1, (2, 0)), (2, (2, 3)), (3, (2, 0)), (0, (1, 2)), (2, (2, 3)), (1, (1, 3)), (2, (1, 3)), (2, (2, 3)), (3, (2, 2)), (2, (2, 3)), (2, (1, 0)), (0, (1, 1)), (2, (2, 1)), (1, (2, 2)), (2, (2, 0)), (1, (2, 0)), (3, (2, 3)), (2, (1, 0)), (3, (2, 1)), (1, (1, 3)), (1, (1, 3)), (0, (1, 1)), (1, (1, 0)), (2, (2, 3)), (2, (1, 0)), (0, (2, 1)), (3, (2, 1)), (0, (1, 2)), (0, (1, 3)), (3, (1, 3)), (2, (1, 2)), (3, (1, 2)), (0, (2, 1)), (2, (2, 2)), (0, (1, 1)), (2, (2, 3)), (0, (1, 3)), (1, (1, 3)), (1, (2, 0)), (2, (1, 0)), (0, (2

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [130]:
def get_random_tup():
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):
  return [get_random_tup() for i in range(count)]

def MAP(t: Tuple):
  return (t[0], t[1])

def tetta(values):
  return sum(values)

def REDUCE(key, values: Iterator[NamedTuple]):
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)

map_out = list(map(lambda x: MAP(x), RECORDREADER(100)))
print("MAP:", map_out)

shuffle_out = group_by_key(map_out)
shuffle_out = list(shuffle_out)
print("Shuffle:", shuffle_out)

reduce_out = list(map(lambda x: REDUCE(*x), shuffle_out))
print("Reduce:", reduce_out)

MAP: [(2, 0), (3, 2), (3, 3), (1, 0), (0, 3), (2, 1), (2, 2), (0, 1), (3, 0), (0, 1), (1, 0), (1, 0), (2, 1), (1, 1), (1, 0), (2, 1), (1, 3), (1, 3), (0, 3), (3, 1), (3, 0), (0, 2), (3, 0), (3, 0), (2, 1), (0, 1), (2, 2), (1, 3), (2, 2), (2, 3), (0, 0), (3, 3), (3, 2), (3, 3), (0, 2), (3, 0), (1, 1), (3, 1), (0, 2), (0, 0), (0, 3), (3, 3), (2, 1), (2, 1), (0, 3), (3, 2), (3, 3), (2, 2), (3, 0), (2, 3), (3, 1), (0, 0), (3, 3), (1, 2), (0, 3), (3, 1), (3, 1), (3, 1), (3, 0), (0, 3), (2, 3), (0, 2), (2, 2), (2, 1), (1, 3), (2, 0), (3, 2), (1, 3), (0, 1), (2, 0), (0, 0), (1, 0), (2, 0), (0, 1), (2, 2), (3, 2), (1, 0), (0, 3), (1, 1), (0, 0), (1, 1), (2, 3), (0, 0), (2, 0), (0, 0), (2, 2), (2, 0), (1, 2), (3, 1), (3, 0), (0, 2), (0, 0), (0, 1), (0, 0), (0, 2), (1, 3), (1, 1), (1, 0), (0, 2), (1, 3)]
Shuffle: [(2, [0, 1, 2, 1, 1, 1, 2, 2, 3, 1, 1, 2, 3, 3, 2, 1, 0, 0, 0, 2, 3, 0, 2, 0]), (3, [2, 3, 0, 1, 0, 0, 0, 3, 2, 3, 0, 1, 3, 2, 3, 0, 1, 3, 1, 1, 1, 0, 2, 2, 1, 0]), (1, [0, 0, 0, 1, 0, 

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [134]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [139]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [141]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [142]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [155]:
import numpy as np
I = 4
J = 6
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
ref_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j])

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k])

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)

def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)

def GET_JOINED():
  for j in joined:
    yield j


joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(ref_solution, asmatrix(solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [176]:
I = 2
J = 3
K = 4*10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):

  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))
  yield second_mat

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
  for j in joined:
    print("join-part", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2

partitioned_out = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_out]

print(joined)
mul_out = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_out]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.007087209458049902), ((0, 1), 0.0020955852417110727), ((0, 2), 0.02710910639729179), ((0, 3), 0.008864124761888857), ((0, 4), 0.019579884164634385), ((0, 5), 0.010826338599294346), ((0, 6), 0.014120606831797052), ((0, 7), 0.01020225359623361), ((0, 8), 0.028497727178153062), ((0, 9), 0.020108307291192232), ((0, 10), 0.012080963849314914), ((0, 11), 0.026111209235602176), ((0, 12), 0.026792516103470564), ((0, 13), 0.021892479674505715), ((0, 14), 0.01437994893104167), ((0, 15), 0.0008222347820925993), ((0, 16), 0.018680201438278365), ((0, 17), 0.0009446101231139221), ((0, 18), 0.013302560301022181), ((0, 19), 0.004516011477449531), ((0, 20), 0.018086005021297976), ((0, 21), 0.010764112066918092), ((0, 22), 0.009403792416672734), ((0, 23), 0.002615553857765342), ((0, 24), 0.025294561311259074), ((0, 25), 0.01970288671492708), ((0, 26), 0.01923003747073442), ((0, 27), 0.009710986075224838), ((0, 28), 0.014437049915372275), ((0

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [181]:
I = 2
J = 3
K = 4*10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1

  if mat_num == 0:
    yield (j, (mat_num, i, w))

  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():

  for j in joined:
    print("join-part", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []

for p in pre_result:

  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.7011604684448118), ((0, 1), 0.18975510935525958), ((0, 2), 0.23738178143124705), ((0, 3), 0.6512765707028215), ((0, 4), 0.10420419646816508), ((0, 5), 0.6278678661160831), ((0, 6), 0.30288538540262444), ((0, 7), 0.11149222375874322), ((0, 8), 0.18459529380932196), ((0, 9), 0.8332633170038758), ((0, 10), 0.8107711930894654), ((0, 11), 0.12358753376815662), ((0, 12), 0.8547137713123574), ((0, 13), 0.7598895526551793), ((0, 14), 0.5642155626363611), ((0, 15), 0.3258193990400772), ((0, 16), 0.049781148593306276), ((0, 17), 0.309088527818102), ((0, 18), 0.022356631090946624), ((0, 19), 0.02059045749555236), ((0, 20), 0.5163088527583863), ((0, 21), 0.13329130659548527), ((0, 22), 0.31482785150256604), ((0, 23), 0.7130666673162585), ((0, 24), 0.006212632115637883), ((0, 25), 0.4084514022245116), ((0, 26), 0.36253710261404776), ((0, 27), 0.0005851565850484897), ((0, 28), 0.4379022896972274), ((0, 29), 0.5364970273211388), ((0, 30),

True